In [146]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import re
import time
from tqdm import tqdm_notebook as tqdm
from datetime import datetime, timedelta

driver = webdriver.Chrome()
start_url = "http://bet.hkjc.com/football/default.aspx?url=/football/schedule/schedule.aspx&lang=en"
driver.get(start_url)
driver.switch_to_frame(driver.find_element_by_tag_name("iframe"))

from_date = "01"
from_month = "08"
from_year = "2016"

to_date = "31"
to_month = "05"
to_year = "2017"

with open('./2016-2017/team_instances_table.json','r') as fp:
    team_table = json.load(fp)

with open('./2016-2017/match_instances_table.json','r') as fp:
    match_table = json.load(fp)

In [147]:
def reloadIframe(webdriver):
    webdriver.switch_to_default_content()
    webdriver.switch_to_frame(webdriver.find_element_by_tag_name("iframe"))

In [149]:
results_dividends_button = driver.find_element_by_xpath("//*[@id=\"betToolsTxt_2\"]")
results_dividends_button.click()
match_results_button = driver.find_element_by_xpath("//*[@id=\"betToolsMenu_2\"]/table/tbody/tr/td/table/tbody/tr[1]/td[2]/a")
match_results_button.click()
time.sleep(2)

jc_match_table = {}

for team_id, team_info in team_table.items():
    jc_team = team_info['jc_name']
    while True:
        try:
            reloadIframe(driver)

            # Select desired start date
            from_date_list = Select(driver.find_element_by_xpath("//*[@id=\"ctl00_cm_srch_DDL_Fday\"]"))
            from_month_list = Select(driver.find_element_by_xpath("//*[@id=\"ctl00_cm_srch_DDL_Fmonth\"]"))
            from_year_list = Select(driver.find_element_by_xpath("//*[@id=\"ctl00_cm_srch_DDL_Fyear\"]"))

            change_from_date = from_date_list.select_by_value(from_date)
            change_from_month = from_month_list.select_by_value(from_month)
            change_from_year = from_year_list.select_by_value(from_year)
            
            # Select desired end date
            to_date_list = Select(driver.find_element_by_xpath("//*[@id=\"ctl00_cm_srch_DDL_Tday\"]"))
            to_month_list = Select(driver.find_element_by_xpath("//*[@id=\"ctl00_cm_srch_DDL_Tmonth\"]"))
            to_year_list = Select(driver.find_element_by_xpath("//*[@id=\"ctl00_cm_srch_DDL_Tyear\"]"))

            change_to_date = to_date_list.select_by_value(to_date)
            change_to_month = to_month_list.select_by_value(to_month)
            change_to_year = to_year_list.select_by_value(to_year)
            
            # Select desired JC team name
            select_team = Select(driver.find_element_by_xpath("//*[@id=\"ctl00_cm_srch_DDL_TeamList\"]"))
            select_team.select_by_visible_text(jc_team)
            
            # Click "search"
            driver.find_element_by_xpath("//*[@id=\"btn_search\"]").click()
            time.sleep(1)
            
            # Map each match to Whoscored match table and record the JC odds report link
            reloadIframe(driver)
            
            page_a_tag = driver.find_elements_by_xpath("//*[@id=\"footballmaincontent\"]/table/tbody/tr[4]/td/div/div[2]/a")
            page_cnt = 1
            while True:
                all_matches = driver.find_elements_by_xpath("//*[@id=\"ctl00_cm_tdMatches\"]/table/tbody/tr")
                series_a_matches = []
                
                for match in all_matches:
                    matchLeague = match.find_element_by_class_name("matchLeague").find_element_by_tag_name('img').get_attribute("title")
                    if matchLeague == "Italian Division 1":
                        series_a_matches.append(match)
                print("{}:{}, {}".format(jc_team, page_cnt, len(series_a_matches)))
                
                for match in series_a_matches:
                    matchTeam = match.find_element_by_class_name("matchTeam").text
                    print(matchTeam)
                    home_team = matchTeam.split(" vs ")[0]
                    away_team = matchTeam.split(" vs ")[1]
                    
                    matchDate = match.find_element_by_class_name("matchDate").text
                    #jc_match_date = datetime.strptime(matchDate, "%d/%m/%Y")
                    
                    matchOdds = match.find_element_by_class_name("matchOdds").find_element_by_tag_name("a").get_attribute("href")
                    
                    jc_match_id = "{},{},{}".format(home_team,away_team,matchDate)
                    
                    if jc_match_id not in jc_match_table:
                        jc_match_table[jc_match_id] = {}
                        jc_match_table[jc_match_id]["odds_link"] = matchOdds
                    
                    
                if page_a_tag[-1].text != "Next":
                    break
                
                page_a_tag[-1].click()
                time.sleep(1)
                page_cnt += 1
                reloadIframe(driver)
                while True:
                    try:
                        page_a_tag = driver.find_elements_by_xpath("//*[@id=\"footballmaincontent\"]/table/tbody/tr[4]/td/div/div[2]/a")
                        break
                    except Exception as e:
                        print("Finding next page button...")
                        print(e)
                        time.sleep(1)
                        
            break
            
        except Exception as e:
            print(e)
            time.sleep(1)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="ctl00_cm_srch_DDL_Fday"]"}
  (Session info: chrome=65.0.3325.162)
  (Driver info: chromedriver=2.36.540470 (e522d04694c7ebea4ba8821272dbef4f9b818c91),platform=Windows NT 6.1.7601 SP1 x86_64)

Napoli:1, 16
Sampdoria vs Napoli
Napoli vs Fiorentina
AC Torino vs Napoli
Napoli vs Cagliari
Inter Milan vs Napoli
Sassuolo vs Napoli
Napoli vs Udinese
Lazio vs Napoli
Napoli vs Juventus
Empoli vs Napoli
Napoli vs Crotone
Roma vs Napoli
Napoli vs Atalanta
Chievo vs Napoli
Napoli vs Genoa
Bologna vs Napoli
Message: stale element reference: element is not attached to the page document
  (Session info: chrome=65.0.3325.162)
  (Driver info: chromedriver=2.36.540470 (e522d04694c7ebea4ba8821272dbef4f9b818c91),platform=Windows NT 6.1.7601 SP1 x86_64)

Napoli:1, 16
Sampdoria vs Napoli
Napoli vs Fiorentina
AC Torino vs Napoli
Napoli vs Cagliari
Inter Milan vs Napoli
Sassuolo vs Napoli
Napoli vs Udinese
Lazio vs Napol

KeyboardInterrupt: 

In [139]:
matchTeam.split(' vs ')[0]

'Sampdoria'

In [76]:
page_buttons[1].click()

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=65.0.3325.162)
  (Driver info: chromedriver=2.36.540470 (e522d04694c7ebea4ba8821272dbef4f9b818c91),platform=Windows NT 6.1.7601 SP1 x86_64)


In [57]:
from_date_list.find_element_by_xpath("/option[1]")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/option[1]"}
  (Session info: chrome=64.0.3282.186)
  (Driver info: chromedriver=2.36.540470 (e522d04694c7ebea4ba8821272dbef4f9b818c91),platform=Windows NT 6.1.7601 SP1 x86_64)


In [124]:
ws_date = datetime.strptime("Sun, 22-Jan-17, 14:00", "%a, %d-%b-%y, %H:%M")

In [126]:
ws_date + timedelta(hours=8)

datetime.datetime(2017, 1, 22, 22, 0)

In [101]:
ws_date - timedelta(hours=8)

datetime.datetime(2017, 2, 25, 16, 0)

In [10]:
import json

with open('./2016-2017/team_instances_table.json','r') as fp:
    team_table = json.load(fp)

with open('./2016-2017/match_instances_table.json','r') as fp:
    match_table = json.load(fp)

In [44]:
with open('./2016-2017/team_instances_table.json', 'w') as fp:
    json.dump(team_table, fp, indent=4)

In [12]:
match_table

{'1115149': {'away_team_id': 77,
  'half_time_result': '0 : 3',
  'home_team_id': 300,
  'match_datetime': 'Sun, 21-Aug-16, 19:45',
  'match_link': 'https://www.whoscored.com/Matches/1115149/Live/Italy-Serie-A-2016-2017-Atalanta-Lazio',
  'result': '3 : 4'},
 '1115150': {'away_team_id': 779,
  'half_time_result': '0 : 0',
  'home_team_id': 71,
  'match_datetime': 'Sun, 21-Aug-16, 19:45',
  'match_link': 'https://www.whoscored.com/Matches/1115150/Live/Italy-Serie-A-2016-2017-Bologna-Crotone',
  'result': '1 : 0'},
 '1115151': {'away_team_id': 75,
  'half_time_result': '0 : 0',
  'home_team_id': 267,
  'match_datetime': 'Sun, 21-Aug-16, 19:45',
  'match_link': 'https://www.whoscored.com/Matches/1115151/Live/Italy-Serie-A-2016-2017-Chievo-Inter',
  'result': '2 : 0'},
 '1115152': {'away_team_id': 271,
  'half_time_result': '0 : 1',
  'home_team_id': 272,
  'match_datetime': 'Sun, 21-Aug-16, 19:45',
  'match_link': 'https://www.whoscored.com/Matches/1115152/Live/Italy-Serie-A-2016-2017-Emp